In [2]:
train = fread("train.csv", na.strings = c(""," ", NA), stringsAsFactors = T)
test = fread("test.csv", na.strings = c(""," " ,NA), stringsAsFactors = T)

Read 532428 rows and 45 (of 45) columns from 0.134 GB file in 00:00:06
Read 354951 rows and 44 (of 44) columns from 0.089 GB file in 00:00:04


In [3]:
summary(train)

   member_id          loan_amnt      funded_amnt    funded_amnt_inv
 Min.   :   70473   Min.   :  500   Min.   :  500   Min.   :    0  
 1st Qu.:10866882   1st Qu.: 8000   1st Qu.: 8000   1st Qu.: 8000  
 Median :37095895   Median :13000   Median :13000   Median :13000  
 Mean   :35005472   Mean   :14758   Mean   :14744   Mean   :14705  
 3rd Qu.:58489201   3rd Qu.:20000   3rd Qu.:20000   3rd Qu.:20000  
 Max.   :73544841   Max.   :35000   Max.   :35000   Max.   :35000  
                                                                   
        term           batch_enrolled      int_rate     grade     
 36 months:372793   BAT2252229: 18791   Min.   : 5.32   A: 89107  
 60 months:159635   BAT3873588: 17839   1st Qu.: 9.99   B:152713  
                    BAT2803411: 17111   Median :12.99   C:147499  
                    BAT2078974: 14859   Mean   :13.24   D: 83567  
                    BAT1586599: 14463   3rd Qu.:16.20   E: 42495  
                    (Other)   :258137   Max.   :28.99 

In [4]:
summary(test)

   member_id          loan_amnt      funded_amnt    funded_amnt_inv
 Min.   :   70626   Min.   :  500   Min.   :  500   Min.   :    0  
 1st Qu.:10889411   1st Qu.: 8000   1st Qu.: 8000   1st Qu.: 8000  
 Median :37086503   Median :13000   Median :13000   Median :13000  
 Mean   :34996354   Mean   :14752   Mean   :14738   Mean   :14699  
 3rd Qu.:58448923   3rd Qu.:20000   3rd Qu.:20000   3rd Qu.:20000  
 Max.   :73544827   Max.   :35000   Max.   :35000   Max.   :35000  
                                                                   
        term           batch_enrolled      int_rate     grade     
 36 months:248332   BAT5260863:  9972   Min.   : 5.32   A: 59095  
 60 months:106619   BAT5662637:  9379   1st Qu.: 9.99   B:101822  
                    BAT6004464:  9000   Median :12.99   C: 98361  
                    BAT6248271:  7957   Mean   :13.25   D: 55975  
                    BAT4106493:  7694   3rd Qu.:16.20   E: 28210  
                    (Other)   :137342   Max.   :28.99 

In [5]:
table(train$loan_status)


     0      1 
406601 125827 

In [6]:
test$loan_status = NA
total = rbindlist(list(train, test), fill = T)

In [7]:
summary(total)

   member_id          loan_amnt      funded_amnt    funded_amnt_inv
 Min.   :   70473   Min.   :  500   Min.   :  500   Min.   :    0  
 1st Qu.:10877134   1st Qu.: 8000   1st Qu.: 8000   1st Qu.: 8000  
 Median :37095283   Median :13000   Median :13000   Median :13000  
 Mean   :35001825   Mean   :14755   Mean   :14742   Mean   :14702  
 3rd Qu.:58471347   3rd Qu.:20000   3rd Qu.:20000   3rd Qu.:20000  
 Max.   :73544841   Max.   :35000   Max.   :35000   Max.   :35000  
                                                                   
        term           batch_enrolled      int_rate     grade     
 36 months:621125   BAT2252229: 18791   Min.   : 5.32   A:148202  
 60 months:266254   BAT3873588: 17839   1st Qu.: 9.99   B:254535  
                    BAT2803411: 17111   Median :12.99   C:245860  
                    BAT2078974: 14859   Mean   :13.25   D:139542  
                    BAT1586599: 14463   3rd Qu.:16.20   E: 70705  
                    (Other)   :439481   Max.   :28.99 

In [8]:
## As a naive model, let's drop some columns first which have high NA's or seems unrelevent
total$member_id = NULL
total$batch_enrolled = NULL
total$desc = NULL
total$zip_code = NULL
total$mths_since_last_delinq = NULL
total$mths_since_last_record = NULL
total$mths_since_last_major_derog = NULL

In [9]:
table(total$verification_status_joint, total$loan_status)

                 
                    0   1
  Not Verified    170   0
  Source Verified  32   1
  Verified        102   0

In [10]:
## Many values are missing plus all are indicating that loan was approved(0 means that person was not defaulter),
## so we might as well remove it
total$verification_status_joint = NULL

In [11]:
str(total$sub_grade)

 Factor w/ 35 levels "A1","A2","A3",..: 23 9 4 20 7 5 4 6 14 19 ...


In [12]:
str(total$emp_title)

 Factor w/ 289207 levels "\tAdv Mtr Proj Fld Rep",..: 166903 68596 46756 154078 77456 40887 144323 15022 78093 118027 ...


In [13]:
str(total$emp_length)

 Factor w/ 12 levels "1 year","10+ years",..: 10 11 3 2 2 3 6 9 8 3 ...


In [14]:
table(total$home_ownership, total$loan_status)

          
                0      1
  ANY           2      1
  MORTGAGE 202344  63596
  NONE          8     28
  OTHER        27     90
  OWN       41737  10927
  RENT     162483  51185

In [15]:
prop.table(table(total$home_ownership, total$loan_status),1)

          
                   0         1
  ANY      0.6666667 0.3333333
  MORTGAGE 0.7608634 0.2391366
  NONE     0.2222222 0.7777778
  OTHER    0.2307692 0.7692308
  OWN      0.7925148 0.2074852
  RENT     0.7604461 0.2395539

In [16]:
sum(total$annual_inc>50000, na.rm = T)

[1] 596240

In [17]:
sum(total$annual_inc>500000, na.rm = T)

[1] 1032

In [18]:
table(total$pymnt_plan, total$loan_status)

   
         0      1
  n 406595 125825
  y      6      2

In [19]:
## almost all values are 'n' so it won't be much helpful in predicting the model
total$pymnt_plan = NULL

In [20]:
str(total$purpose)

 Factor w/ 14 levels "car","credit_card",..: 3 5 3 3 2 3 3 2 5 3 ...


In [21]:
str(total$addr_state)

 Factor w/ 51 levels "AK","AL","AR",..: 10 21 36 46 5 16 5 2 5 18 ...


In [22]:
table(total$addr_state)


    AK     AL     AR     AZ     CA     CO     CT     DC     DE     FL     GA 
  2205  11200   6640  20412 129517  18807  13531   2432   2511  60935  29085 
    HI     IA     ID     IL     IN     KS     KY     LA     MA     MD     ME 
  4570     14     12  35476  13789   7926   8550  10587  20593  21031    525 
    MI     MN     MO     MS     MT     NC     ND     NE     NH     NJ     NM 
 22985  15957  14207   3819   2558  24720    479   1176   4294  33256   4939 
    NV     NY     OH     OK     OR     PA     RI     SC     SD     TN     TX 
 12443  74086  29631   8085  10893  31393   3893  10639   1815  12887  71138 
    UT     VA     VT     WA     WI     WV     WY 
  6264  26255   1797  19434  11574   4386   2028 

In [23]:
head(total$emp_title)

[1] clerk                             Human Resources Specialist       
[3] Driver                            Us office of Personnel Management
[5] LAUSD-HOLLYWOOD HIGH SCHOOL       Design Consultant                
289207 Levels: \tAdv Mtr Proj Fld Rep ... zulily

In [24]:
## replace the missing values with median of the data
total$annual_inc[is.na(total$annual_inc)] = 75028
total$delinq_2yrs[is.na(total$delinq_2yrs)] = 0
total$inq_last_6mths[is.na(total$inq_last_6mths)] = 0
total$open_acc[is.na(total$open_acc)] = 11
total$pub_rec[is.na(total$pub_rec)] = 0
total$revol_util[is.na(total$revol_util)] = 56
total$total_acc[is.na(total$total_acc)] = 24
total$collections_12_mths_ex_med[is.na(total$collections_12_mths_ex_med)] = 0

In [25]:
total$acc_now_delinq[is.na(total$acc_now_delinq)] = 0

In [26]:
str(total$emp_title)

 Factor w/ 289207 levels "\tAdv Mtr Proj Fld Rep",..: 166903 68596 46756 154078 77456 40887 144323 15022 78093 118027 ...


In [27]:
## there are too many levels, we need to reduce them
emp_title = data.frame(table(total$emp_title))

In [28]:
sum(emp_title$Freq==1)

[1] 236538

In [29]:
sum(emp_title$Freq==2)

[1] 25238

In [30]:
emp_title1 = emp_title[emp_title$Freq==1,]

In [31]:
summary(total$emp_title)

Teacher                  Manager         Registered Nurse 
                   13807                    11638                     5691 
                   Owner                       RN               Supervisor 
                    5512                     5491                     5414 
                   Sales          Project Manager                   Driver 
                    4457                     4087                     3722 
          Office Manager                 Director          General Manager 
                    3615                     3328                     3281 
                 manager                  teacher                    owner 
                    3247                     2979                     2904 
                Engineer                President           Vice President 
                    2743                     2679                     2560 
                  driver                 Attorney       Operations Manager 
                    2479                     2231                     2166 
Administrative Assistant               Accountant            Sales Manager 
                    2148                     2114                     1901 
              supervisor                    sales          Account Manager 
                    1806                     1799                     1786 
          Police Officer      Executive Assistant               Technician 
                    1782                     1690                     1673 
                 Analyst            Store Manager                    Nurse 
                    1616                     1568                     1476 
            truck driver             Truck Driver                Paralegal 
                    1464                     1453                     1377 
       Software Engineer               Consultant        Assistant Manager 
                    1367                     1292                     1285 
              Controller           Branch Manager          Program Manager 
                    1253                     1229                     1222 
       Account Executive            Administrator                   Server 
                    1163                     1160                     1160 
                Mechanic                Principal         Business Analyst 
                    1140                     1118                     1043 
        registered nurse         Registered nurse              Electrician 
                    1026                     1024                     1009 
      Executive Director                      CEO                Professor 
                     995                      993                      980 
              IT Manager                  Foreman                 mechanic 
                     963                      937                      906 
          Superintendent         Customer Service                Associate 
                     898                      876                      841 
  Director of Operations                Secretary          Legal Assistant 
                     837                      827                      819 
                Operator         District Manager                      LPN 
                     819                      777                      765 
              Pharmacist        Financial Analyst        Financial Advisor 
                     751                      744                      738 
               Physician             Truck driver              Maintenance 
                     732                      720                      708 
                 Officer                    nurse               technician 
                     676                      676                      675 
               Machinist       Accounting Manager                    Clerk 
                     669                      663                      660 
               Bartender               Instructor                

In [32]:
total$emp_title = as.character(total$emp_title)

In [33]:
## Replace the 'emp_title' which appears only once by 'emp_type_1'
total$emp_title[total$emp_title %in% emp_title1$Var1] = 'emp_type_1'

In [34]:
emp_title2 = emp_title[emp_title$Freq > 1 & emp_title$Freq <= 100,]

In [35]:
## Replace the 'emp_title' which appears with frequence between 1 and 100 by 'emp_type_2'
total$emp_title[total$emp_title %in% emp_title2$Var1] = 'emp_type_2'

In [36]:
total$emp_title = as.factor(total$emp_title)

In [38]:
model.rpart = rpart(loan_status ~ ., data = head(total,nrow(train)), method = "class")

In [39]:
pred.rpart = predict(model.rpart, tail(total, nrow(test)), type = "prob")

In [41]:
submit = data.frame(member_id = test$member_id, loan_status = pred.rpart[,2])

In [42]:
## Now there are some cases where probability may come out to be absolute 0 or 1 which is not accepted because as per the
## competition's rule, values must be between 0 and 1. So we need to deal with such cases.
submit$loan_status[submit$loan_status == 0] = 0.001
submit$loan_status[submit$loan_status == 1] = 0.999

In [43]:
write.csv(submit, "sub.rpart.csv", row.names = F)